# Team Assignment - Intentionally Blank

The following libraries should be installed before proceeding further:
- pandas
- numpy

In [1]:
#Import all necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In addition to the available datasets from the team assignment the original dataset is read for additional information about each ride. The additional information contains the geographical coordinates for the start position and the end position of each ride.

In [2]:
# Reading in weather and philadelphia_2017 data
df_weather = pd.read_csv("data/weather_hourly_philadelphia.csv")
df_philadelphia_2017 = pd.read_csv("data/philadelphia_2017.csv")

# Additionally read the philadelphia data from the official website
data_philadelphia_2017_Q1 = pd.read_csv("data/indego_Q1_2017.csv")
data_philadelphia_2017_Q2 = pd.read_csv("data/indego_Q2_2017.csv")
data_philadelphia_2017_Q3 = pd.read_csv("data/indego_Q3_2017.csv")
data_philadelphia_2017_Q4 = pd.read_csv("data/indego_Q4_2017.csv")

## Preparing and cleaning data

### Preparing and cleaning the philadelphia datasets

In [3]:
# Concatenate all quarters of philadelphia data and convert them to a dataframe
df_philadelphia_website = pd.concat([data_philadelphia_2017_Q1, data_philadelphia_2017_Q2, data_philadelphia_2017_Q3, data_philadelphia_2017_Q4], ignore_index=True)

In [4]:
# Dropping the columns duration, start_station_id, end_station_id, plan_duration, trip_route_category, passholder_type, start_station, end_station
df_philadelphia_website.drop(['duration', 'start_station_id', 'end_station_id', 'plan_duration', 'trip_route_category', 'passholder_type', 'start_station', 'end_station', 'trip_id'], axis=1, inplace=True)
df_philadelphia_website

#Changing the type of start_time, end_time (df_philadelphia_2017 and df_philadelphia_website) and date_time (df_weather) to datetime type from pandas
df_philadelphia_website.loc[:,'start_time'] = pd.to_datetime(df_philadelphia_website['start_time'])
df_philadelphia_website.loc[:,'end_time'] = pd.to_datetime(df_philadelphia_website['end_time'])

df_philadelphia_2017.loc[:,'start_time'] = pd.to_datetime(df_philadelphia_2017['start_time'])
df_philadelphia_2017.loc[:,"end_time"] = pd.to_datetime(df_philadelphia_2017["end_time"])

#Sort dataframes by their corresponding time column
df_philadelphia_website.sort_values(["start_time"], inplace = True)
df_philadelphia_2017.sort_values(["start_time"], inplace = True)


#Resetting the indexes
df_philadelphia_website.reset_index(drop = True, inplace = True)
df_philadelphia_2017.reset_index(drop = True, inplace = True)

Before we further clean the dataset for philadelphia its important to mention that we first deleted the irrelevant stations because some duplicate rows happen to have the same start time, end time and bike ids but different start and end stations like the "Virtual Station". Therefore we deleted those stations first to avoid deleting useful records. In the following example you can see that deleting the duplicates first could lead to deleting useful records and keeping irrelevant records which are deleted either way later.

In [5]:
df_philadelphia_2017[df_philadelphia_2017.duplicated(subset=['start_time', 'end_time', 'bike_id'], keep=False)].head(6)

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name
255753,2017-06-05 15:26:00,2017-06-05 15:27:00,3152,3152,11907,Indego30,"40th & Baltimore, Trolley Portal","40th & Baltimore, Trolley Portal"
255755,2017-06-05 15:26:00,2017-06-05 15:27:00,3000,3000,11907,Indego30,Virtual Station,Virtual Station
256143,2017-06-05 17:26:00,2017-06-05 17:31:00,3152,3024,11907,Walk-up,"40th & Baltimore, Trolley Portal","43rd & Chester, Clark Park"
256148,2017-06-05 17:26:00,2017-06-05 17:31:00,3000,3024,11907,Walk-up,Virtual Station,"43rd & Chester, Clark Park"
258113,2017-06-06 16:02:00,2017-06-06 16:33:00,3004,3000,5314,Indego30,Municipal Services Building Plaza,Virtual Station
258115,2017-06-06 16:02:00,2017-06-06 16:33:00,3004,3161,5314,Indego30,Municipal Services Building Plaza,30th Street Station East


The "3083" station does not exist as a real station. Furthermore the "Virtual Station" is used by staff to check in or check out a bike remotely for an event or when the bike was not properly checked in or out. Therefore we check the number of rows where one of those stations occur and delete those rows afterwards.

For further information please check the official site about the data and the station table [here](https://www.rideindego.com/about/data/).

In [6]:
print(r'Number of rows in df_philadelphia_2017 with "Virtual Station" and "3083" in the start station or end station: ',
 df_philadelphia_2017[(df_philadelphia_2017["start_station_name"] == "Virtual Station") | (df_philadelphia_2017["end_station_name"] == "Virtual Station") |
  (df_philadelphia_2017["start_station_name"] == "3083") | (df_philadelphia_2017["end_station_name"] == "3083")
  ].shape[0])

Number of rows in df_philadelphia_2017 with "Virtual Station" and "3083" in the start station or end station:  6512


In [7]:
# Deleting "Virtual Station" and "3083" from the start_station_name and end_station_name columns
df_philadelphia_2017 = df_philadelphia_2017[(df_philadelphia_2017["start_station_name"] != "3083") & (df_philadelphia_2017["end_station_name"] != "3083") ]
df_philadelphia_2017 = df_philadelphia_2017[(df_philadelphia_2017["start_station_name"] != "Virtual Station") & (df_philadelphia_2017["end_station_name"] != "Virtual Station") ]

display(df_philadelphia_2017[(df_philadelphia_2017["start_station_name"] == "Virtual Station") | (df_philadelphia_2017["end_station_name"] == "Virtual Station") | (df_philadelphia_2017["start_station_name"] == "3083") | (df_philadelphia_2017["end_station_name"] == "3083")])

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name


The "bike_id" should be unique to one bike. No trip with the same bike_id should start and end on the same time interval. 

In [8]:
#Checking for any duplicates in philadelphia data
print("Number of duplicates in df_philadelphia_2017: ", df_philadelphia_2017.duplicated(subset=['start_time', 'end_time', 'bike_id']).sum())
print("Number of duplicates in df_philadelphia_website: ", df_philadelphia_website.duplicated(subset=['start_time', 'end_time', 'bike_id']).sum())

Number of duplicates in df_philadelphia_2017:  55
Number of duplicates in df_philadelphia_website:  68


In [9]:
#Dropping all duplicates for the philadelphia data depending on the start_time, end_time and bike_id
df_philadelphia_2017.drop_duplicates(subset=["start_time", "end_time", "bike_id"], keep="first", inplace=True)
df_philadelphia_website.drop_duplicates(subset=["start_time", "end_time", "bike_id"], keep="first", inplace=True)

display(df_philadelphia_2017[df_philadelphia_2017.duplicated(subset=['start_time', 'end_time', 'bike_id'])].head(10))
display(df_philadelphia_website[df_philadelphia_website.duplicated(subset=['start_time', 'end_time', 'bike_id'])].head(10))

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name


,start_time,end_time,start_lat,start_lon,end_lat,end_lon,bike_id


In [10]:
# Joining df_philadelphia_2017 with df_philadelphia_full on start_time, end_time, bike_id and dropping duplicate columns
df_philadelphia_2017_joined = df_philadelphia_2017.merge(df_philadelphia_website, on=["start_time", "end_time", "bike_id"], how="left")

df_philadelphia_2017_joined

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,start_lat,start_lon,end_lat,end_lon
0,2017-01-01 00:05:00,2017-01-01 00:16:00,3046,3041,5347,Indego30,2nd & Market,"Girard Station, MFL",39.950119,-75.144722,39.968491,-75.135460
1,2017-01-01 00:21:00,2017-01-01 00:57:00,3110,3054,3364,Walk-up,Del. River Trail & Penn St.,Rodin Museum,39.961750,-75.136414,39.962502,-75.174202
2,2017-01-01 00:22:00,2017-01-01 00:57:00,3110,3054,2536,Walk-up,Del. River Trail & Penn St.,Rodin Museum,39.961750,-75.136414,39.962502,-75.174202
3,2017-01-01 00:27:00,2017-01-01 00:39:00,3041,3005,5176,Indego30,"Girard Station, MFL","Welcome Park, NPS",39.968491,-75.135460,39.947330,-75.144028
4,2017-01-01 00:28:00,2017-01-01 00:36:00,3047,3124,5370,Walk-up,"Independence Mall, NPS",Race Street Pier,39.950710,-75.149208,39.952950,-75.139793
...,...,...,...,...,...,...,...,...,...,...,...,...
782335,2017-12-31 23:05:00,2017-12-31 23:33:00,3070,3124,3708,Indego30,"Spring Garden Station, MFL",Race Street Pier,39.960621,-75.139832,39.952950,-75.139793
782336,2017-12-31 23:11:00,2018-01-01 11:03:00,3107,3165,5117,Indego30,33rd & Reservoir,24th & Race SRT,39.982029,-75.188660,39.958191,-75.178200
782337,2017-12-31 23:18:00,2017-12-31 23:25:00,3033,3046,11933,Indego30,10th & Chestnut,2nd & Market,39.950050,-75.156723,39.950119,-75.144722
782338,2017-12-31 23:39:00,2017-12-31 23:40:00,3163,3163,6725,Indego30,25th & Locust,25th & Locust,39.949741,-75.180969,39.949741,-75.180969


In [11]:
#Checking for missing values in philadelphia data
display(df_philadelphia_2017_joined[df_philadelphia_2017_joined.isnull().any(axis = 1)])

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,start_lat,start_lon,end_lat,end_lon
255784,2017-06-06 16:02:00,2017-06-06 16:33:00,3004,3161,5314,Indego30,Municipal Services Building Plaza,30th Street Station East,39.953781,-75.163742,NaN,NaN
613438,2017-10-05 08:35:00,2017-10-05 08:47:00,3111,3107,5306,Indego30,"Parkside & Belmont, Case Building",33rd & Reservoir,NaN,NaN,39.982029,-75.18866
613848,2017-10-05 10:31:00,2017-10-05 10:53:00,3095,3111,5329,Indego30,29th & Diamond,"Parkside & Belmont, Case Building",39.987709,-75.180519,NaN,NaN


In [12]:
df_philadelphia_2017_joined.dropna(inplace = True)	

In [ ]:
#Adding hour, month, weekday and trip duration(based on start and end time) to the table
df_philadelphia_2017_joined.loc[:,"trip_duration"] = df_philadelphia_2017_joined["end_time"] - df_philadelphia_2017_joined["start_time"] 
df_philadelphia_2017_joined.loc[:,"date"] = df_philadelphia_2017_joined["start_time"].dt.date
df_philadelphia_2017_joined.loc[:,"month"] = df_philadelphia_2017_joined["start_time"].dt.month
df_philadelphia_2017_joined.loc[:,"week"] = df_philadelphia_2017_joined["start_time"].dt.isocalendar().week
df_philadelphia_2017_joined.loc[:,"weekday"] = df_philadelphia_2017_joined["start_time"].dt.weekday
df_philadelphia_2017_joined.loc[:,"day"] = df_philadelphia_2017_joined["start_time"].dt.day
df_philadelphia_2017_joined.loc[:,"hour"] = df_philadelphia_2017_joined["start_time"].dt.hour
df_philadelphia_2017_joined

We also remove trips which are shorter than one minute and longer than one day because the bike sharing service does not allow those trips normally.

In [ ]:
df_philadelphia_2017_joined[df_philadelphia_2017_joined["trip_duration"] < "0 days 00:01:00"]

In [ ]:
df_philadelphia_2017_joined = df_philadelphia_2017_joined[df_philadelphia_2017_joined["trip_duration"] >= "0 days 00:01:00"]
df_philadelphia_2017_joined[df_philadelphia_2017_joined["trip_duration"] < "0 days 00:01:00"]

In [ ]:
df_philadelphia_2017_joined[df_philadelphia_2017_joined["trip_duration"] > "1 days"]

In [ ]:
df_philadelphia_2017_joined = df_philadelphia_2017_joined[df_philadelphia_2017_joined["trip_duration"] <= "1 days"]
df_philadelphia_2017_joined[df_philadelphia_2017_joined["trip_duration"] > "1 days"]

In [ ]:
df_philadelphia_cleaned = df_philadelphia_2017_joined.copy()
df_philadelphia_cleaned

### Preparing and cleaning the weather dataset

After preparing and cleaning the philadelphia dataset we need to do most of the procedure on the weather dataset.

In [ ]:
#Changing the type of date_time (df_weather) to datetime type from pandas
df_weather.loc[:, "date_time"] = pd.to_datetime(df_weather["date_time"])

#Sort the dataframe by their corresponding time column
df_weather.sort_values(["date_time"], inplace = True)

#Reset the index
df_weather.reset_index(drop = True, inplace =True)

First of all we check for duplicate rows in the weather data and delete them if they occur because we do not need multiple rows multiple times.

In [ ]:
#Checking for any duplicates in weather data
print("Number of duplicates in df_weather: ", df_weather.duplicated().sum())

In [ ]:
#Dropping all duplicates for the weather data
df_weather.drop_duplicates(subset= ["date_time"],inplace = True)

display(df_weather[df_weather.duplicated()])

We also check for null values in the weather data and delete them. It is important to note that we later add additional rows with null values but with the missing time intervals because we can interpolate them with the interpolate function in pandas.

In [ ]:
#Checking for missing values in weather data
display(df_weather[df_weather.isnull().any(axis = 1)])

In [ ]:
df_weather.dropna(inplace = True)

Since the weather data consists of data from the beginning of 2015 until the end of 2019, we want to have as well only the weather data for 2017.

In [ ]:
#Selecting data from the beginning of 2017 till the end of 2017
df_weather_2017 = df_weather[(df_weather["date_time"]>= "2017-01-01 00:00:00") & (df_weather["date_time"]< "2018-01-01 00:00:00" )]

In [ ]:
#Add missing intervals in the weather data
df_weather_2017.set_index("date_time", inplace = True)
df_weather_2017 = df_weather_2017.resample("H").asfreq()

The minimum temperature and maximum temperature will be interpolated linearly. The precipitation value is interpolated by taking one of the nearest existing values.

In [ ]:
# Interpolate missing values in the weather data
df_weather_2017["min_temp"].interpolate(method = "linear", inplace = True)
df_weather_2017["max_temp"].interpolate(method = "linear", inplace = True)
df_weather_2017["precip"].interpolate(method = "pad", inplace = True)
df_weather_2017.reset_index(drop=False, inplace = True)

# Check if null values in the weather data still exist
display(df_weather_2017[df_weather_2017.isnull().any(axis = 1)])
display(df_weather_2017)

All dataframes for reference:
* **Philadelphia data**
   * *df_philadelphia_cleaned*
* **Weather data**
   * *df_weather_2017*

## Descriptive Analytics

### Temporal Demand Patterns and Seasonality

#### Analysis of Rental during the day

First count the started rentals for each hour of the day for the whole dataset and vizualize it in a histogram.

In [ ]:
#Calculate the number of rentals in each hour (accumulated over the year)
df = df_philadelphia_cleaned
df.groupby(["hour"])["hour"].count().reset_index(name="n_rentals_within_hour")

In [ ]:
#Calculate the mean
mean = df.groupby(["hour"])["hour"].count().mean()
mean

In [ ]:
plt.hist(df["hour"], bins=range(0,25,1), color="green")
plt.axhline(mean , color = 'r', linestyle = '--')
plt.xlabel("Hour")
plt.ylabel("Number of rentals started")
plt.title("Development of started rentals during the day")
plt.xticks(range(0,25,2))
plt.grid(True)
min_xlim, max_xlim = plt.xlim()
plt.text(min_xlim*(-0.2), mean*1.1, 'Mean: {:.2f}'.format(mean))
plt.show()

To get average values we calculate the number of days for which we have data in our dataset and calculate the avarage number of rentals started within each hour in a day.

IMPORTANT: The average values only provide meaningful results if we assume that we have all meaningful rental transactions that actually occurred included in our data set or at least equally distributed missing transaction for each hour.

In [ ]:
#Check the number of days for which we have data in our set
number_days = len(df["date"].unique())
number_days

In [ ]:
#Calculate average values for each hour
df_avg_per_hour = df.groupby(["hour"])["hour"].count().divide(number_days).reset_index(name="avg_n_rentals_within_hour")
df_avg_per_hour

In [ ]:
#Calculate mean
mean= df_avg_per_hour["avg_n_rentals_within_hour"].mean()
mean

In [ ]:
plt.bar(
        df_avg_per_hour["hour"], 
        df_avg_per_hour["avg_n_rentals_within_hour"], 
        color="green"
)
plt.axhline(mean , color = 'r', linestyle = '--')
plt.xlabel("Hour")
plt.ylabel("Average number of rentals started within hour")
plt.title("Development of avg started rentals during the day")
min_xlim, max_xlim = plt.xlim()
plt.text(min_xlim*(-0.2), mean*1.1, 'Mean: {:.2f}'.format(mean))
plt.grid(True)
plt.show()

In the plots above we only considered the starting times of the rentals. To also considere the endtime, we calculate how many bikes in the respective hours were IN USE accumulated over the year. (e.g. rental from 0 o'clock to 2:45 would count as a usage in hour 0, 1 and 2). In the process of calculation we also have to considere that bike rentals can range over two days.

In [ ]:
#Calculate average number of bikes in use
df["end_time_hour"] = df["end_time"].dt.hour
df["end_time_date"] = df["end_time"].dt.date
df_values = pd.DataFrame()
for i in range(0,24,1):
    df_values[f"{i}"]= (((df["hour"] <= i) & (df["end_time_hour"] >= i)) 
                        | ((df["end_time_hour"] >= i) & (df["date"] < df["end_time_date"])))

df_sum = df_values.apply(lambda x: x.sum()/number_days).reset_index(name="avg_n_of_bikes_in_use").rename(columns={"index": "hour"})
df_sum

In [ ]:
#Calculate mean
mean = df_sum["avg_n_of_bikes_in_use"].mean()
mean

In [ ]:
plt.bar(df_sum["hour"], df_sum["avg_n_of_bikes_in_use"], color="green")
plt.axhline(mean , color = 'r', linestyle = '--')
plt.xlabel("Hour")
plt.ylabel("Average Number of Bikes in Use")
plt.title("Bikes in Use by Hours of the Day")
min_xlim, max_xlim = plt.xlim()
plt.text(min_xlim*(-0.2), mean*1.1, 'Mean: {:.2f}'.format(mean))
plt.grid(True)
plt.show()

##### Analysis and Interpretation of the Results:

The plots above all show that we have two peaks of bike rentals in the day. One peak demand at 8 am and the other peak at 5 pm. This peak can be a result of the rush-hour traffic. At these times most people are on their way to work/school or on their way back home. The demand around these times ( 8 am and 5 pm ) are also above average. Besides that, it also becomes clear that the bikes are mostly used during the day. During the night the demand is significantly below average (from 9 pm until 6 am).

#### Analysis of Rental during the week

First count the started rentals for each weekday of the day for the whole dataset and vizualize it in a histogram.

In [ ]:
#Calculate accumulated rentals per weekday
df.groupby(["weekday"])["weekday"].count().reset_index(name="n_rentals_within_weekday")

The mapping: {0: "Monday", 1: "Tuesday", 2: "Wednesday", 3: "Thursday", 4: "Friday",5: "Saturday",6: "Sunday"}

In [ ]:
#Calculate mean
mean = df.groupby(["weekday"])["weekday"].count().mean()
mean

In [ ]:
plt.hist(df["weekday"], bins=range(0,8,1), color="green")
plt.axhline(mean , color = 'r', linestyle = '--')
plt.xlabel("Weekday")
plt.ylabel("Total Number of Rentals Started on Weekday")
plt.title("Development of Started Rentals During the Week")
plt.xticks(range(0,8,1))
min_xlim, max_xlim = plt.xlim()
plt.text(min_xlim*(-0.1), mean*1.1, 'Mean: {:.2f}'.format(mean))
plt.grid(True)
plt.show()

To get average values we calculate the number of weeks for which we have data in our dataset and calculate the avarage number of rentals started within each weekday.

IMPORTANT ASSUMPTION: The average values only provide meaningful results if we assume that we have all rental transactions that actually occurred included in our data set or at least equally distributed missing data points for each weekday. Otherwise the average values would provide misleading information. If we for example would have missing rental data for a specific weekday for several weeks, dividing by the total number of weeks would result to false average value for this weekday!

In [ ]:
#Check number of weeks we have in our set
number_weeks = len(df["week"].unique())
number_weeks

In [ ]:
#Calculate average values per weekday
df_average_per_weekday = df.groupby(["weekday"])["weekday"].count().divide(number_weeks).reset_index(name="avg_number_started_rentals")
df_average_per_weekday

In [ ]:
#Calculate mean
mean = df_average_per_weekday["avg_number_started_rentals"].mean()
mean

In [ ]:
plt.bar(
        df_average_per_weekday["weekday"], 
        df_average_per_weekday["avg_number_started_rentals"], 
        color="green"
)
plt.axhline(mean, color="r", linestyle="--")
plt.xlabel("Weekday")
plt.ylabel("Average Number of Rentals Started per Weekday")
plt.title("Development of Started Rentals During the Week")
min_xlim, max_xlim = plt.xlim()
plt.text(min_xlim*(-0.1), mean*1.1, 'Mean: {:.2f}'.format(mean))
plt.grid(True)
plt.show()

##### Analysis and Interpretation of Results:

The Bike rental demand is the lowest at the weekends (Saturdays and Sundays). The demand these days is significantly below average. The demand on Mondays is slightly below the average. We have a peak demand on Wednesdays. This development can possibly be rooted in the fact that most people don't work/ go to school or university on the weekends. This could mean that a big share of people who use the bike rental service are people who are on their way to work, school, or university.

#### Analysis of Rental during the Year

First count the started rentals for each month of the year for the whole dataset and vizualize it in a histogram.
Average values can not be calculated since we only have data of one year.

In [ ]:
#Calculate number of rentals per month
df.groupby(["month"])["month"].count().reset_index(name="n_of_retals_in_month")

In [ ]:
#Calculate mean
mean = df.groupby(["month"])["month"].count().mean()
mean

In [ ]:
plt.hist(df["month"], bins=range(0,14,1), color="green")
plt.axhline(mean, color="r", linestyle="--")
plt.xlabel("Weekday")
plt.ylabel("Total Number of Rentals Started in Month")
plt.title("Development of Started Rentals during Month")
plt.xticks(range(1,13,1))
min_xlim, max_xlim = plt.xlim()
plt.text(min_xlim*(-0.1), mean*(1.06), "Mean: {:.2f}".format(mean))
plt.grid(True)
plt.show()

In [ ]:
df_weather = df_weather_2017
df_weather["date"] = df_weather["date_time"].dt.date
df_weather["hour"] = df_weather["date_time"].dt.hour

#Join weather data with rental data and calculate average temperatures
df_merge = pd.merge(df, df_weather, how="left", left_on=["date", "hour"], right_on=["date", "hour"])
df_merge["avg_temp"] = df_merge[["max_temp", "min_temp"]].mean(axis="columns")

In [ ]:
#Calculate average temperatures for each month
df_temp_month = df_merge.groupby("month")["avg_temp"].mean().reset_index(name="avg_temp")
df_temp_month

In [ ]:
#Plot number of rentals together with average temperature
#Red dots would be the average temperature for each month
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
df_months = df.groupby(["month"])["month"].count().reset_index(name="n_of_retals_in_month")
ax1.bar(df_months["month"], df_months["n_of_retals_in_month"], color="green")
plt.xlabel("Weekday")
ax1.set_ylabel("Total Number of Rentals Started in Month")
plt.title("Development of Started Rentals and Temperature")
plt.xticks(range(1,13,1))
ax2.scatter(df_temp_month["month"], df_temp_month["avg_temp"], color="red")
ax2.set_ylabel("Average Temperature")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(df_temp_month["avg_temp"], df_months["n_of_retals_in_month"], color="red")
plt.xlabel("Average Temperature")
plt.ylabel("Bike Rental Demand")
plt.title("Development of Bike Rental Demand dependant on Average Temperature")
plt.grid(True)
plt.show()

##### Analysis and Interpretation of the Results:

The development of bike rental demand has similar development as the development of the average temperature during the year. 
The only difference can be seen at the peak. The average temperature has its peak in July whereas the bike rental demand has its peak in August. However, the difference in the bike rental demand in July and August is not that high. Another effect that could have an influence on the bike rental demand is the number of tourists. According to the annual report of Philadelphia, the number of tourists was the highest for the second and third quarters of the year (April to September). In these months the bike rental demand was always very close (April) or above the overall average of the bike rental demand. 
Especially for the month of December, January, February, and March the bike rental demand is clearly below the average.

Development of the tourist numbers according to the annual report: 
- Q1: 7 million
- Q2: 13.1 million
- Q3: 13.1 million
- Q4: 10.1 million

### Key Performance Indicators (KPIs)

#### Rental Durations

Rental durations are closely tied to revenue because the revenue model of the bike sharing company is based on the rental time/ duration thus its relevant to observe the rental duration to get information on the company's performance.

In [ ]:
#Calculate the sum of rental durations for each month
#Calculate the average rental duration per rental for each month
df = df_philadelphia_cleaned
df_duration_monthly = df.groupby("month")["trip_duration"].sum().reset_index(name="sum_duration_bike_rentals")
df_duration_monthly["duration_in_hours"] = df_duration_monthly["sum_duration_bike_rentals"].dt.total_seconds().divide(60*60)
df_duration_monthly["duration_in_minutes"] = df_duration_monthly["sum_duration_bike_rentals"].dt.total_seconds().divide(60)
df_duration_monthly["number_of_rentals"] = df.groupby("month")["month"].count().reset_index(drop=True)
df_duration_monthly["avg_duration_per_rental_in_min"] = df_duration_monthly["duration_in_minutes"] / df_duration_monthly["number_of_rentals"]
df_duration_monthly

In [ ]:
#Vizualize development of summed durations
plt.bar(df_duration_monthly["month"],
       df_duration_monthly["duration_in_hours"],
       color="green")
plt.grid(True)
plt.xticks(range(1,13,1))
plt.xlabel("Months")
plt.ylabel("Sum of Rental Durations")
plt.title("Development of the Summed Rental Durations")
plt.show()

In [ ]:
#Vizualize development of average durations for each month
plt.bar(df_duration_monthly["month"],
       df_duration_monthly["avg_duration_per_rental_in_min"],
       color="green")
plt.grid(True)
plt.xticks(range(1,13,1))
plt.xlabel("Months")
plt.ylabel("Avg Duration per Rental in min")
plt.title("Development of Average Rental Duration")
plt.show()

In [ ]:
#Calculate the sum of rental durations for each week
#Calculate the average rental duration per rental for each week
df_duration_weekly = df.groupby("week")["trip_duration"].sum().reset_index(name="sum_duration_bike_rentals")
df_duration_weekly["duration_in_hours"] = df_duration_weekly["sum_duration_bike_rentals"].dt.total_seconds().divide(60*60)
df_duration_weekly["duration_in_minutes"] = df_duration_weekly["sum_duration_bike_rentals"].dt.total_seconds().divide(60)
df_duration_weekly["number_of_rentals"] = df.groupby("week")["week"].count().reset_index(drop=True)
df_duration_weekly["avg_duration_per_rental_in_min"] = df_duration_weekly["duration_in_minutes"] / df_duration_weekly["number_of_rentals"]
df_duration_weekly.head()

In [ ]:
#Vizualize development of summed durations
plt.rcParams["figure.figsize"] = (20,8)
plt.plot(df_duration_weekly["week"],
       df_duration_weekly["duration_in_hours"],
       color="green")
plt.grid(True)
plt.xticks(range(1,53,1))
plt.xlabel("Weeks")
plt.ylabel("Sum of Rental Durations")
plt.title("Development of the Summed Rental Durations")
plt.show()

In [ ]:
#Vizualize development of average durations for each week
plt.plot(df_duration_weekly["week"],
       df_duration_weekly["avg_duration_per_rental_in_min"],
       color="green")
plt.grid(True)
plt.xticks(range(1,53,1))
plt.xlabel("Weeks")
plt.ylabel("Average of Rental Durations")
plt.title("Development of the Average Rental Durations")
plt.show()

##### Interpretation of the Results

- The summed rental durations have a similar development as the summed number of rentals 
- Our bikes are more used in the summer months especially from June to October
- This can be seen in the monthly as well as the weekly aggregation 
- The average rental duration per ride lies between 15 and 22 minutes with the average rental duration being especially high from April to July

##### Rental Duration Dependant on Weather

In [ ]:
df_weather = df_weather_2017
df_weather["date"] = df_weather["date_time"].dt.date
df_weather["hour"] = df_weather["date_time"].dt.hour

#Join weather data with rental data and calculate average temperatures
df_merge = pd.merge(df, df_weather, how="left", left_on=["date", "hour"], right_on=["date", "hour"])
df_merge["avg_temp"] = df_merge[["max_temp", "min_temp"]].mean(axis="columns")

In [ ]:
df_duration_daily = df_merge.groupby("date").agg({"trip_duration": lambda x: x.sum(), 
                                                  "precip": lambda x: x.sum(), 
                                                  "avg_temp": lambda x: x.mean()})
df_duration_daily["duration_in_hours"] = df_duration_daily["trip_duration"].dt.total_seconds().divide(60*60)
df_duration_daily["duration_in_minutes"] = df_duration_daily["trip_duration"].dt.total_seconds().divide(60)
df_duration_daily["number_of_rentals"] = df.groupby("date")["date"].count()
df_duration_daily["avg_duration_per_rental_in_min"] = df_duration_daily["duration_in_minutes"] / df_duration_daily["number_of_rentals"]
df_duration_daily_precip = df_duration_daily[df_duration_daily["precip"] > 0]
df_duration_daily_no_precip = df_duration_daily[df_duration_daily["precip"] == 0]
df_duration_daily

In [ ]:
#Calculate average summed rental duration per day
avg_summed_duration_precip = df_duration_daily_precip["duration_in_hours"].mean()
avg_summed_duration_no_precip = df_duration_daily_no_precip["duration_in_hours"].mean()
print(f"{avg_summed_duration_precip= }\n{avg_summed_duration_no_precip= }")

In [ ]:
#Calculate average rental duration per rental
avg_duration_per_rental_precip = df_duration_daily_precip["avg_duration_per_rental_in_min"].mean()
avg_duration_per_rental_no_precip = df_duration_daily_no_precip["avg_duration_per_rental_in_min"].mean()
print(f"{avg_duration_per_rental_precip= }\n{avg_duration_per_rental_no_precip= }")

In [ ]:
#Calculate average rental duration and average summed rental duration per day dependen on temperature
df_duration_daily["temp_bin"] = pd.cut(df_duration_daily.avg_temp, 10)
df_temp_bins = df_duration_daily.groupby("temp_bin").agg({"duration_in_hours": lambda x: x.mean(), 
                                          "avg_duration_per_rental_in_min": lambda x: x.mean()}).reset_index()
df_temp_bins["temp_bin"] = df_temp_bins["temp_bin"].astype(str)
df_temp_bins

In [ ]:
plt.rcParams["figure.figsize"] = (20,8)
plt.bar(df_temp_bins["temp_bin"], df_temp_bins["duration_in_hours"], color="green")
plt.xlabel("Temperature")
plt.ylabel("Average Summed Trip Durations per Day in Hours")
plt.title("Development of Average Summed Trip Durations per Day Dependent on Temperature")
plt.grid(True)
plt.show()

In [ ]:
plt.bar(df_temp_bins["temp_bin"], df_temp_bins["avg_duration_per_rental_in_min"], color="green")
plt.xlabel("Temperature")
plt.ylabel("Average Trip Duration in Minutes")
plt.title("Development of Average Trip Durations Dependent on Temperature")
plt.grid(True)
plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]
plt.show()

##### Interpretation of the Results

- When additionally taking weather data into account, we can observe that the average summed rental durations per day are lower on days with precipitation. This is also the case for the average rental duration per day
- Besides that, the average summed rental durations per day also increase with the increase of the temperature
- With the average rental duration per trip we only slightly see this effect (there is not a continuous increase and not a very extreme one as with summed rental durations)

#### Bike Utilization

Bike utilization gives us information on how much the different bikes are used.
This information is relevant to get information about possible unutilized /unused bikes which are maybe actually not needed (unused ressources/capacitiy).
Moreover we analyse how many unique bikes at which time and at which weather condition.

In [ ]:
#approximate how many individual bikes there are (we have no information about total number of bikes)
number_of_bikes = len(df["bike_id"].unique())
total_number_of_rentals = len(df) # each row is a rental
print(f"{number_of_bikes=}, {total_number_of_rentals=}")

In [ ]:
#Calculate total number of rides for each bike over the whole year
df_bike_ride_count = df.groupby("bike_id")["start_time"].count().reset_index(name="total_number_of_rides").sort_values(by="total_number_of_rides", ascending=False).reset_index(drop=True)
df_bike_ride_count["cumulative_ride_count"] = df_bike_ride_count["total_number_of_rides"].cumsum()
df_bike_ride_count["percentage_number_of_bikes"] = df_bike_ride_count.index.to_series().add(1).divide(number_of_bikes)
df_bike_ride_count["percentage_ride_count"] = df_bike_ride_count["cumulative_ride_count"].divide(total_number_of_rentals)
df_bike_ride_count

In [ ]:
plt.rcParams["figure.figsize"] = (10,5)
plt.plot(df_bike_ride_count.index.to_series(), df_bike_ride_count["total_number_of_rides"], color="green")
plt.grid(True)
plt.ylabel("Total Number of Rides")
plt.xlabel("Bike Count")
plt.title("Total Number of Rides per Bike")
plt.show()

In [ ]:
plt.plot(df_bike_ride_count.index.to_series(), df_bike_ride_count["cumulative_ride_count"], color="green")
plt.grid(True)
plt.xlabel("Bike Count")
plt.ylabel("Number of Rentals")
plt.title("Cumulative Number of Rentals")
plt.show()

In [ ]:
plt.plot(df_bike_ride_count["percentage_number_of_bikes"], df_bike_ride_count["percentage_ride_count"], 
         color="green")
plt.grid(True)
plt.xlabel("Number of Bikes in Percent")
plt.ylabel("Number of Rentals in Percent")
plt.title("Number of Bikes and Rentals in Percent")
plt.show()

In [ ]:
#Utilization by month
df_util_month = df.groupby("month")["bike_id"].nunique().reset_index(name="number_of_bikes_used")
df_util_month["utilization"] = df_util_month["number_of_bikes_used"].divide(number_of_bikes)
df_util_month

In [ ]:
plt.bar(df_util_month["month"], df_util_month["number_of_bikes_used"], color="green")
plt.xlabel("Months")
plt.ylabel("Number of Used Bikes")
plt.title("Utilization of Bikes per Month")
plt.grid(True)
plt.show()

In [ ]:
plt.bar(df_util_month["month"], df_util_month["utilization"], color="green")
plt.xlabel("Months")
plt.ylabel("Utilization in Percent")
plt.title("Utilization of Bikes per Month")
plt.grid(True)
plt.show()

It seems that our initial assumption of the total number of bikes could be untrue for the different months.
It could be possible that bikes are added in some months. For example in Mai there is an sudden increase of the bike utilization. 
We will check it in the following steps.

In [ ]:
set(df[df["month"]==5]["bike_id"].unique()) - set(df[df["month"]==4]["bike_id"].unique())

 It seems that bikes above 11700 are added in Mai.

In [ ]:
#Check for bikes with BikeID above 11700 for months 1 to 4
for i in range(1,5):
    print((df[df["month"]==i]["bike_id"] > 11700).any())

In [ ]:
#Get the highest BikeID for each month
df.groupby("month")["bike_id"].max()

In [ ]:
#Weekly values
df_bike_count_weekly = df.groupby("week")["bike_id"].nunique().reset_index(name="number_of_bikes")

In [ ]:
plt.plot(df_bike_count_weekly["week"], df_bike_count_weekly["number_of_bikes"], color="green")
plt.xlabel("Week")
plt.ylabel("Number of Used Bikes")
plt.title("Number of Used Bikes per Week")
plt.xticks(range(0,53,2))
plt.grid(True)
plt.show()

In [ ]:
#Daily values
df_bike_count_daily = df.groupby(["date"])["bike_id"].nunique().reset_index(name="number_of_bikes")
df_bike_count_daily

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.plot(df_bike_count_daily["date"], df_bike_count_daily["number_of_bikes"], color="green")
plt.grid(True)
plt.yticks(range(0,1000,100))
plt.xlabel("Day")
plt.ylabel("Number of Bikes")
plt.title("Number of Bikes Used per Day")
plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]
plt.show()

In [ ]:
#Average bike utilization by weekday
df_bike_count_daily["date"] = pd.to_datetime(df_bike_count_daily["date"])
df_bike_count_daily["weekday"] = df_bike_count_daily["date"].dt.weekday
df_weekday = df_bike_count_daily.groupby("weekday")["number_of_bikes"].mean().reset_index(name="avg_n_bikes")
df_weekday

In [ ]:
plt.bar(df_weekday["weekday"], df_weekday["avg_n_bikes"], color="green")
plt.xlabel("Weekday")
plt.ylabel("Number of Bikes")
plt.title("Average Number of Bikes per Weekday")
plt.grid(True)
plt.show()

In [ ]:
#Average bike utilization by hour of day
df_bike_count_hourly = df.groupby(["date", "hour"])["bike_id"].nunique().reset_index(name="number_of_bikes")
df_hour_of_day = df_bike_count_hourly.groupby("hour")["number_of_bikes"].mean().reset_index(name="avg_n_bikes")
df_hour_of_day

In [ ]:
plt.bar(df_hour_of_day["hour"], df_hour_of_day["avg_n_bikes"], color="green")
plt.xlabel("Hour")
plt.ylabel("Number of Bikes")
plt.title("Average Number of Used Bikes per Hour")
plt.xticks(range(0,24,1))
plt.grid(True)
plt.show()

##### Interpretation of the Results

- We approximated which number of total bikes there could be in the fleet 
- We  found a total of 1249 individual bikes in the data set 
- However, there are likely bikes added over time, since some bikeID's (especially higher ones) only occur starting at a certain month in the year
- This fact has to be considered when analyzing the results

- When analyzing the utilization of the individual bikes we can observe that only a few bikes (about five) are used significantly more than the others. And about two hundred bikes are used less than most bikes. But for most bikes, as seen in the graph the utilization does not vary too much. Besides that, because some bikes are likely to be added over time, it makes sense that these bikes were not used as much as the rest of the bikes.

- The bike utilization per month (considering a total number of bikes of 1249) shows that it was likely that bikes were added for example in Mai. Since we can see a sudden increase in the utilization (in the months before the utilization was quite the same)

- When analyzing the average number of bikes used per weekday it becomes clear that there are on average more bikes used during the week than during the weekends. Similar to the development of the number of rides.
- The development during the day also shows similar results to the development of the number of rides. We have clear peaks at 8 am and 5 pm.

##### Bike Utilization Dependent on Weather

In [ ]:
#Calculate average number of bikes used on hour with precipitation vs. one with none
df_hourly = df_merge.groupby(["date", "hour"]).agg({"bike_id": lambda x: x.nunique(), 
                                                    "precip": lambda x: x.sum(),
                                                   "avg_temp": lambda x: x.mean()})
df_hourly = df_hourly.rename(columns={"bike_id": "n_bikes_used"})
df_no_precip_hourly = df_hourly[df_hourly["precip"] == 0]
df_precip_hourly = df_hourly[df_hourly["precip"] > 0]
print(f"with precip: {df_precip_hourly['n_bikes_used'].mean()} avg bikes used per hour\nwithout: {df_no_precip_hourly['n_bikes_used'].mean()} avg bikes used per hour")

In [ ]:
#Calculate average number of bikes used on hour dependent on temperature
df_hourly["temp_bin"] = pd.cut(df_hourly["avg_temp"], 10)
df_temp_bins_hourly = df_hourly.groupby("temp_bin")["n_bikes_used"].mean().reset_index(name="avg_n_bikes_per_hour")
df_temp_bins_hourly["temp_bin"] = df_temp_bins_hourly["temp_bin"].astype(str)
df_temp_bins_hourly

In [ ]:
plt.rcParams["figure.figsize"] = (20,8)
plt.bar(df_temp_bins_hourly["temp_bin"], df_temp_bins_hourly["avg_n_bikes_per_hour"], color="green")
plt.xlabel("Temperature")
plt.ylabel("Average Number of Bikes Used per Hour")
plt.title("Average Number of Bikes Used per Hour Dependant on Temperature")
plt.grid(True)
plt.show()

In [ ]:
#Calculate average number of bikes used on day with precipitation vs. one with none
df_daily = df_merge.groupby("date").agg({"bike_id": lambda x: x.nunique(), 
                                         "precip": lambda x: x.sum(),
                                         "avg_temp": lambda x: x.mean()})
df_daily = df_daily.rename(columns={"bike_id": "n_bikes_used"})
df_no_precip_daily = df_daily[df_daily["precip"] == 0]
df_precip_daily = df_daily[df_daily["precip"] > 0]
print(f"with precip: {df_precip_daily['n_bikes_used'].mean()} avg bikes used per day\nwithout: {df_no_precip_daily['n_bikes_used'].mean()} avg bikes used per day")

In [ ]:
#Calculate average number of bikes used on day dependent on temperature
df_daily["temp_bin"] = pd.cut(df_daily["avg_temp"], 10)
df_daily

In [ ]:
df_temp_bins = df_daily.groupby("temp_bin")["n_bikes_used"].mean().reset_index(name="avg_n_bikes_per_day")
df_temp_bins["temp_bin"] = df_temp_bins["temp_bin"].astype(str)
df_temp_bins

In [ ]:
plt.bar(df_temp_bins["temp_bin"], df_temp_bins["avg_n_bikes_per_day"], color="green")
plt.xlabel("Temperature")
plt.ylabel("Average Number of Bikes Used per Day")
plt.title("Average Number of Bikes Used per Day Dependant on Temperature")
plt.grid(True)
plt.show()

##### Interpretation of the Results

The average number of bikes used per hour is lower when precipitationis recorded. Besides that, we see a steady increase in the average number of bikes used per hour with the increase in the temperature. This is also the case for the average number of bikes used per day.

## Predictive Analytics

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

##### Features
- day_of_week 
- hour_of_day 
- month 
- avg_temp (min and max temp don't differ much)
- precipitation

We chose day of the week as a feature because it became clear in the descriptive analytics part that the weekday has an effect on the bike-rental demand. Especially on the weekends the demand is much lower. It would also be possible to convert the variable into a binary one (weekend) to only check if the respective day falls into the weekend. However, we would then lose some information because even on some days during the week the demand is slightly higher than on other days. Besides that Du et. al. also used the feature day of the week to forecast bike rental demand.  

The descriptive analytics part also showed that the hour of the day has also an effect on the rental demand. Especially during the night the demand is low and we have peaks at the rush hours (8 am and 5 pm).  

Besides that, the demand seems to differ dependent on the month. For the summer months, we clearly see an increase in the bike rental demand.   To use this information we decided to include it as a feature in our models.
The min and max temperature for one hour do not differ much and thus have a high correlation which we wanted to remove. Thus we decided to work with the average temperature for the respective hours.  

Besides that precipitation seems to also have an effect on the demand. The number of rentals and also the summed rental durations are higher for no precipitation records.

Du et al. paper from Standford Univesity: http://cs229.stanford.edu/proj2014/Jimmy%20Du,%20Rolland%20He,%20Zhivko%20Zhechev,%20Forecasting%20Bike%20Rental%20Demand.pdf

### Prepare Dataframe

In [ ]:
df = df_philadelphia_cleaned.copy()
df = df.set_index("start_time")
df = df.loc[:, ["day", "month", "weekday", "hour", "end_time"]]
df = df.resample("H").agg({
    "day" : "max",
    "month": "max",
    "weekday": "max",
    "hour": "max",
    "end_time": "count"
})
df = df.rename(columns={"end_time": "demand"})
df.index.names = ["time"]
display(df)

In [ ]:
df = df[~df.apply(lambda x: x.isnull().any(), axis=1)]

In [ ]:
#merge the weather data
df_weather = df_weather_2017.copy()
df_weather = df_weather.loc[:, ["date_time","max_temp", "min_temp", "precip"]]
df_merge = pd.merge(df, df_weather, left_index=True, right_on="date_time").set_index("date_time")
df_merge["avg_temp"] = df_merge[["max_temp", "min_temp"]].mean(axis=1)
df_merge

In [ ]:
#split into dependent and independent variables
y = df_merge["demand"]
X = df_merge.drop(["demand", "min_temp", "max_temp"], axis=1)

In [ ]:
#create test and train and validation set

# Do a 70-30 split first
X_train_hold, X_test, y_train_hold, y_test = train_test_split(X, y, test_size=0.3,random_state=34)

# now split X_train to achive 50-20-30 split
X_train, X_hold, y_train, y_hold = train_test_split(X_train_hold, y_train_hold, test_size=(0.2/0.7),random_state=34)


#### Testing Linear Regression

In [ ]:
#Plotting function

def plot_prediction(X_tr, y_tr, X_te, y_pr, column):
    plt.figure(figsize = (8,6))
    plt.scatter(X_tr[column], y_tr, marker="x", c='C1')
    plt.xlabel(column)
    plt.ylabel("demand")
    plt.scatter(X_te[column], y_pr, marker="x", c='red')
    plt.legend(['Observed data', 'fitted data'])

In [ ]:
#Metrics function

def get_metrics(y_train, y_pred_train, y_test, y_pred_test):

    return {
            "MAE": {
                    "train": mean_absolute_error(y_train, y_pred_train),
                    "test": mean_absolute_error(y_test, y_pred_test)
                }, 
            "r2": {
                    "train": r2_score(y_train, y_pred_train),
                    "test": r2_score(y_test, y_pred_test)
                }
    }

In [ ]:
#Coefficients

def get_coefficients(model, columns):
        return pd.DataFrame(
                    {
                        "variable" : columns,
                        "coef": model.coef_
                    })
    

In [ ]:
#Regression function

def linear_regression(X_train, y_train, X_test, y_test, plot, column, get_coef=None):   
    #Fitting
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)
    
    #Prediction on testing and training set
    y_pred_test = lin_reg.predict(X_test)
    y_pred_train = lin_reg.predict(X_train)
    y_pred_test[y_pred_test < 0] = 0
    y_pred_train[y_pred_train < 0] = 0
    
    if(plot):
        plot_prediction(X_train, y_train, X_train, y_pred_train, column)
        
    return (get_metrics(y_train, y_pred_train, y_test, y_pred_test),
            get_coefficients(lin_reg, X_train.columns) if get_coef else None,
            y_pred_test,
            y_pred_train)

In [ ]:
metrics, coef, _, __ = linear_regression(X_train_hold, y_train_hold, X_test, y_test, True, "avg_temp", True)
display(metrics, coef)

#### Testing Linear Regression with dummy variables

The usage of linear regression would not yield good results with the feature set we have. The descriptive analytics	part with its plots clearly shows that most features do not have a linear relationship with the dependent variable (bike rental demand). For example, for the feature of hour_of_day, we have two peaks of the bike rental demand which can not be represented bike a linear function. The only feature that seems to have a linear relationship with the dependent variable is the average temperature. We see an increasing demand with increasing temperature. 
But if we see the features day_of_week, hour_of_day, and month as categorical features, we could actually yield much better results. We decided to convert each of these features into multiple dummy variables. By doing so we would generate in the end linear model with multiple intercepts.

This approach has the following drawbacks: 
- we would increase the model complexity by increasing dimensionality
- we would lose the ordering of the features (features are actually ordinary)

In [ ]:
X_train_hold.loc[:,["day", "month","weekday", "hour"]] = X_train_hold.loc[:,["day", "month", "weekday", "hour"]].astype(str)
X_test.loc[:,["day", "month", "weekday", "hour"]] = X_test.loc[:,["day", "month", "weekday", "hour"]].astype(str)
X_train_dum = pd.get_dummies(X_train_hold)
X_test_dum = pd.get_dummies(X_test)

In [ ]:
metrics, coef, _, __ = linear_regression(X_train_dum, y_train_hold, X_test_dum, y_test, True, "avg_temp", True)
display(metrics, coef)

#####  Results:
- the r2 score gives as information about the proportion of the variance of the dependent variable (the bike-rental demand) which can be explained by our model (the independent variables). This in this case for the test set 72.46 % of the variance of the bike rental demand can be explained through our model.
- besides that the means absolute error (how far was our prediction of the bike rental demand on average away from the actual value) is 31.46
- the coefficents for the different dummy variables can be interpreted as the different intercepts.

### Polynomial Regression

As stated above we have some features for which we don’t	have a linear relationship with the dependent variable. This non-linear relationship can be modeled using polynomial features through which we can better approximate the non-linear relationship.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
def poly_regression(X_train, y_train, X_test, y_test, degree, plot=None, column=None):   # method can be r2 or MAE
    
    # initialize model
    poly_reg = PolynomialFeatures(degree)

    # fit and transform
    X_poly = poly_reg.fit_transform(X_train)
    X_poly_test = poly_reg.fit_transform(X_test)
    
    return linear_regression(X_poly, y_train, X_poly_test, y_test, plot, column)
    


def find_poly_regression(X_train, y_train, X_hold, y_hold, max_value):
    
    mae = {"hold": [], "train": []}
    r2 = {"hold": [], "train": []}
    
    array = np.arange(1,max_value +1)
    
    for i in array:
        metrics, _, __,___ = poly_regression(X_train, y_train, X_hold, y_hold, i)
        mae["hold"].append(metrics["MAE"]["test"]) 
        mae["train"].append(metrics["MAE"]["train"])
        r2["hold"].append(metrics["r2"]["test"]) 
        r2["train"].append(metrics["r2"]["train"])
    
    # plot results
    
    # plot results
    plt.figure(figsize = (8,6))
    plt.plot(array, mae["train"], label="train")
    plt.plot(array, mae["hold"], label="hold")
    plt.legend()
    plt.xlabel("Number of polynomials")
    plt.ylabel("MAE")
    plt.show()
    
    plt.figure(figsize = (8,6))
    plt.plot(array, r2["train"], label="train")
    plt.plot(array, r2["hold"], label="hold")
    plt.legend()
    plt.xlabel("Number of polynomials")
    plt.ylabel("r2")
    plt.show()

In [ ]:
find_poly_regression(X_train, y_train, X_hold, y_hold, 5)

In [ ]:
find_poly_regression(X_train, y_train, X_hold, y_hold, 6)

##### Selection of the degree of polynomials
For the polynomial of degree 5 we seem to yield the best results regarding the r2 score and mean absolute error. For a polynomial of degree 6 we can see that the mean absolut error rises and the r2 score drops. 

In [ ]:
metrics, _, y_pred_test, y_pred_train = poly_regression(X_train, y_train, X_test, y_test, 5)
display(metrics)
plot_prediction(X_train, y_train, X_train, y_pred_train, "avg_temp")

##### Results:
- the r2 score gives as information about the proportion of the variance of the dependent variable (the bike-rental demand) which can be explained by our model (the independent variables). This in this case for the test set 64.40 % of the variance of the bike rental demand can be explained through our model.
- besides that the means absolute error (how far was our prediction of the bike rental demand on average away from the actual value) is 35.03

In [ ]:
plot_prediction(X_train, y_train, X_train, y_pred_train, "month")